# qecsim demos

## Simulating error correction with a rotated planar stabilizer code
This demo shows verbosely how to simulate one error correction run. 

| For normal use, the code in this demo is encapsulated in the function:
| `qecsim.app.run_once(code, error_model, decoder, error_probability)`,
| and the simulation of many error correction runs is encapsulated in the function:
| `qecsim.app.run(code, error_model, decoder, error_probability, max_runs, max_failures)`.

Notes:

* Operators can be visualised in binary symplectic form (bsf) or Pauli form, e.g. `[1 1 0|0 1 0] = XYI`.
* The binary symplectic product is denoted by $\odot$ and defined as $A \odot B \equiv A \Lambda B \bmod 2$ where $\Lambda = \left[\begin{matrix} 0 & I \\ I & 0 \end{matrix}\right]$.
* Binary addition is denoted by $\oplus$ and defined as addition modulo 2, or equivalently exclusive-or.

### Initialise the models

In [3]:
%run qsu.ipynb  # color-printing functions
import collections
import itertools
import numpy as np
from qecsim import paulitools as pt
import matplotlib.pyplot as plt
import qecsim
from qecsim import app
from qecsim.models.generic import PhaseFlipErrorModel,DepolarizingErrorModel,BiasedDepolarizingErrorModel
from qecsim.models.planar import PlanarCode, PlanarMPSDecoder
from qecsim.models.rotatedplanar import RotatedPlanarCode, RotatedPlanarMPSDecoder

from _planarmpsdecoder_def import PlanarMPSDecoder_def
from _rotatedplanarmpsdecoder_def import RotatedPlanarMPSDecoder_def
import app_def
import importlib as imp
imp.reload(app_def)
import os, time
import multiprocessing as mp
from functools import partial
from itertools import cycle

# initialise models
my_code = RotatedPlanarCode(7, 7)
my_error_model = DepolarizingErrorModel()
my_decoder = RotatedPlanarMPSDecoder(chi=8)
# print models
print(my_code)
print(my_error_model)
print(my_decoder)

RotatedPlanarCode(7, 7)
DepolarizingErrorModel()
RotatedPlanarMPSDecoder(8, 'c', None)


In [5]:
code = RotatedPlanarCode(6,6)
print(code.site_bounds)
code_index=(0,0)
code.is_z_plaquette(code_index)
def _rotate_q_index(index, code):
    """Convert code site index in format (x, y) to tensor network q-node index in format (r, c)"""
    site_x, site_y = index  # qubit index in (x, y)
    site_r, site_c = code.site_bounds[1] - site_y, site_x  # qubit index in (r, c)
    print(site_r, site_c)
    return code.site_bounds[0] - site_c + site_r, site_r + site_c  # q-node index in (r, c)

_rotate_q_index(code_index, code)

(5, 5)
5 0


(10, 5)

In [6]:
tn_max_r, _ = _rotate_q_index((0, 0), code)
_, tn_max_c = _rotate_q_index((code.site_bounds[0], 0), code)
tn = np.empty((tn_max_r + 1, tn_max_c + 1), dtype=object)
tn.shape

5 0
5 5


(11, 11)

### Generate a random error

In [7]:
import collections
import itertools
import numpy as np
from qecsim import paulitools as pt
import matplotlib.pyplot as plt
import qecsim
from qecsim import app
from qecsim.models.generic import PhaseFlipErrorModel,DepolarizingErrorModel,BiasedDepolarizingErrorModel
from qecsim.models.planar import PlanarCode, PlanarMPSDecoder
from qecsim.models.rotatedplanar import RotatedPlanarCode, RotatedPlanarMPSDecoder

from _planarmpsdecoder_def import PlanarMPSDecoder_def
from _rotatedplanarmpsdecoder_def import RotatedPlanarMPSDecoder_def
import app_def
import importlib as imp
imp.reload(app_def)
import os, time
import multiprocessing as mp
from functools import partial
from itertools import cycle

In [8]:
# set physical error probability to 10%
error_probability = 0.4
bias =0.5
# seed random number generator for repeatability
rng = np.random.default_rng(10)
error_model = BiasedDepolarizingErrorModel(bias,'Z')
chi_val=10
decoder = RotatedPlanarMPSDecoder_def(chi=chi_val)
    
# error: random error based on error probability
error = error_model.generate(code, error_probability, rng)
qsu.print_pauli('error:\n{}'.format(code.new_pauli(error)))

### Evaluate the syndrome
The syndrome is a binary array indicating the stabilizers with which the error does not commute. It is calculated as $syndrome = error \odot stabilisers^T$.

In [31]:
# syndrome: stabilizers that do not commute with the error
syndrome = pt.bsp(error, my_code.stabilizers.T)
qsu.print_pauli('syndrome:\n{}'.format(my_code.ascii_art(syndrome)))

### Find a recovery operation
In this case, the recovery operation is found by contracting a tensor network defined to have the value of the coset :

* Create a sample recovery operation by applying strings of Paulis between syndrome plaquettes and appropriate boundaries.
* Define tensor networks corresponding to the probability of each left coset of the generator group with the sample recovery and logical Pauli operations.
* Contract each tensor network (approximately) to evaluate the coset probabilities.
* Return any recovery from the most probable coset.

In [32]:
# recovery: best match recovery operation based on decoder
recovery = my_decoder.decode(my_code, syndrome)
qsu.print_pauli('recovery:\n{}'.format(my_code.new_pauli(recovery)))

As a sanity check, we expect $recovery \oplus error$ to commute with all stabilizers, i.e. $(recovery \oplus error) \odot stabilisers^T = 0$.

In [33]:
# check recovery ^ error commutes with stabilizers (by construction)
print(pt.bsp(recovery ^ error, my_code.stabilizers.T))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


### Visualise $recovery \oplus error$
Just out of curiosity, we can see what $recovery \oplus error$ looks like. If successful, it should be a product of stabilizer plaquette operators.

In [34]:
# print recovery ^ error (out of curiosity)
qsu.print_pauli('recovery ^ error:\n{}'.format(my_code.new_pauli(recovery ^ error)))

### Test if the recovery operation is successful
The recovery operation is successful iff $recovery \oplus error$ commutes with all logical operators, i.e. $(recovery \oplus error) \odot logicals^T = 0.$

In [35]:
# success iff recovery ^ error commutes with logicals
print(pt.bsp(recovery ^ error, my_code.logicals.T))

[0 0]


Note: The decoder is not guaranteed to find a successful recovery operation. The rotated planar 7 x 7 code has distance $d = 7$ so we can only guarantee to correct errors up to weight $(d - 1)/2=3$.

### Equivalent code in single call
The above demo is equivalent to the following code.

In [4]:
# repeat demo in single call
from qecsim import app
print(app.run_once(my_code, my_error_model, my_decoder, error_probability))

{'error_weight': 7, 'success': True, 'logical_commutations': array([0, 0]), 'custom_values': None}


In [40]:
%matplotlib inline
import collections
import itertools
import numpy as np
from qecsim import paulitools as pt
import matplotlib.pyplot as plt
from qecsim import app
from qecsim.models.generic import PhaseFlipErrorModel,DepolarizingErrorModel
from qecsim.models.planar import PlanarCode, PlanarMPSDecoder
from _planarmpsdecoder_def import PlanarMPSDecoder_def
import app_def
import importlib as imp
imp.reload(app_def)
sizes= range(5,9,2)
codes_and_size = [PlanarCode(*(size,size)) for size in sizes]

In [18]:
codes_and_size

[PlanarCode(5, 5), PlanarCode(7, 7)]

In [19]:
x=np.zeros(5)

In [20]:
x=np.append(x,1)

In [21]:
x

array([0., 0., 0., 0., 0., 1.])

In [23]:
for code_index in itertools.product(range(-1, 5 + 1), range(-1, 5 + 1)):
    print(code_index)

(-1, -1)
(-1, 0)
(-1, 1)
(-1, 2)
(-1, 3)
(-1, 4)
(-1, 5)
(0, -1)
(0, 0)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(0, 5)
(1, -1)
(1, 0)
(1, 1)
(1, 2)
(1, 3)
(1, 4)
(1, 5)
(2, -1)
(2, 0)
(2, 1)
(2, 2)
(2, 3)
(2, 4)
(2, 5)
(3, -1)
(3, 0)
(3, 1)
(3, 2)
(3, 3)
(3, 4)
(3, 5)
(4, -1)
(4, 0)
(4, 1)
(4, 2)
(4, 3)
(4, 4)
(4, 5)
(5, -1)
(5, 0)
(5, 1)
(5, 2)
(5, 3)
(5, 4)
(5, 5)


In [ ]:
import collections
import itertools
import numpy as np
from qecsim import paulitools as pt
import matplotlib.pyplot as plt
import qecsim
from qecsim import app
from qecsim.models.generic import PhaseFlipErrorModel,DepolarizingErrorModel,BiasedDepolarizingErrorModel
from qecsim.models.planar import PlanarCode, PlanarMPSDecoder
from qecsim.models.rotatedplanar import RotatedPlanarCode, RotatedPlanarMPSDecoder

from _planarmpsdecoder_def import PlanarMPSDecoder_def
from _rotatedplanarmpsdecoder_def import RotatedPlanarMPSDecoder_def
import app_def
import importlib as imp
imp.reload(app_def)
import os, time
import multiprocessing as mp
from functools import partial
from itertools import cycle

def parallel_step_p(code,hadamard_mat,error_model, decoder, max_runs,error_probability):
    result= app_def.run_def(code,hadamard_mat, error_model, decoder, error_probability, max_runs)
    return result

# set models
sizes= range(5,10,4)
codes_and_size = [RotatedPlanarCode(*(size,size)) for size in sizes]
bias_list=[30,100000]

code_name="optimal"
code_name="random"
code_name="CSS"
code_name="XZZX"

if (code_name=="random"):
    realizations=50
else:
    realizations=1

# set physical error probabilities
error_probability_min, error_probability_max = 0.05, 0.24
error_probabilities = np.linspace(error_probability_min, error_probability_max, 10)
# set max_runs for each probability
max_runs = 2000

timestr = time.strftime("%Y%m%d-%H%M%S ")   #record current date and time
dirname="./data/"+timestr+code_name
os.mkdir(dirname)     

for bias in bias_list:
    error_model = BiasedDepolarizingErrorModel(bias,'Z')
    chi_val=10
    decoder = RotatedPlanarMPSDecoder_def(chi=chi_val)

    # print run parameters
    print('codes_and_size:', [code.label for code in codes_and_size])
    print('Error model:', error_model.label)
    print('Decoder:', decoder.label)
    print('Error probabilities:', error_probabilities)
    print('Maximum runs:', max_runs)

    pL_list_rand =np.zeros((len(codes_and_size),realizations,len(error_probabilities)))
    std_list_rand=np.zeros((len(codes_and_size),realizations,len(error_probabilities)))

    pL_list =np.zeros((len(codes_and_size),len(error_probabilities)))
    std_list=np.zeros((len(codes_and_size),len(error_probabilities)))
    def _rotate_q_index(index, code):
        """Convert code site index in format (x, y) to tensor network q-node index in format (r, c)"""
        site_x, site_y = index  # qubit index in (x, y)
        site_r, site_c = code.site_bounds[1] - site_y, site_x  # qubit index in (r, c)
        return code.site_bounds[0] - site_c + site_r, site_r + site_c  # q-node index in (r, c)

    for code_index,code in enumerate(codes_and_size):
        tn_max_r, _ = _rotate_q_index((0, 0), code)
        _, tn_max_c = _rotate_q_index((code.site_bounds[0], 0), code)

        hadamard_mat=np.zeros((tn_max_r,tn_max_c))

        if code_name=="random":
            pH=0.5  
            for realization_index in range(realizations):
                for row, col in np.ndindex(hadamard_mat.shape):
                    if ((row%2==0 and col%2==0) or (row%2==1 and col%2==1)):
                        if(np.random.rand(1,1))<pH:
                            hadamard_mat[row,col]=1

                p=mp.Pool()
                func=partial(parallel_step_p,code,hadamard_mat, error_model, decoder, max_runs)
                result=p.map(func, error_probabilities)
                #print(result)
                p.close()
                p.join()
                for i in range(len(result)):
                    pL_list_rand[code_index][realization_index][i]=result[i][0]
                    std_list_rand[code_index][realization_index][i]=result[i][1]
                
            pL_list[code_index] = np.sum(pL_list_rand[code_index],axis=0)/realizations
            std_list[code_index] = np.sum(std_list_rand[code_index],axis=0)/realizations**2

        else:
            if code_name=="XZZX":
                for row, col in np.ndindex(hadamard_mat.shape):
                    if row%2==0 and col%2==0:
                        hadamard_mat[row,col]=1

            if code_name=="optimal":
                d=hadamard_mat.shape[0]
                for row, col in np.ndindex(hadamard_mat.shape):
                    if (row ==0 and col in range(0,d,4)) or (row==d-1 and col in range(2,d-1,4)) or (row%2==1 and col%2==1):
                        hadamard_mat[row,col]=1

            p=mp.Pool()
            func=partial(parallel_step_p,code,hadamard_mat,error_model, decoder, max_runs)
            result=p.map(func, error_probabilities)
            print(result)
            p.close()
            p.join()
            
            for i in range(len(result)):
                pL_list[code_index][i]=result[i][0]   
                std_list[code_index][i]=result[i][1]

        
    plt.figure(figsize=(20,10))
    lines = ["-",":","--","-."]
    linecycler = cycle(lines)
    plt.title('TND at bias='+str(bias)+' and xi='+str(chi_val))
    for sizes_index,size in enumerate(sizes):
        plt.errorbar(error_probabilities,pL_list[sizes_index],std_list[sizes_index])
    plt.xlabel('p')
    plt.ylabel('$p_L$')
    plt.legend(sizes) 
    plt.savefig(dirname+"/threshold_plot_bias_"+str(bias)+".pdf")


In [39]:
rng = np.random.default_rng() if rng is None else rng
size=3
code=PlanarCode(*(size,size+1)) 

n_qubits =code.n_k_d[0]
probability=0.8
error_model = BiasedDepolarizingErrorModel(bias,'Z')
p=error_model.probability_distribution(probability)

p

NameError: name 'rng' is not defined

In [ ]:
    3x4 planar code (H=qubit on horizontal edge, V=qubit on vertical edge):
    ::
      H1---H2---H3---H4
          |   |   |
          V13 V14 V15
          |   |   |
      H5---H6---H7---H8
          |   |   |
          V16 V17 V18
          |   |   |
      H9---H10--H11-H12


In [58]:
code.n_k_d

(18, 1, 3)

In [59]:
error_pauli = ''.join(rng.choice(
            ('I', 'X', 'Y', 'Z'),
            size=n_qubits,
            p=error_model.probability_distribution(probability)
        ))
error_pauli

'YZXZZZXIZZYXZZYZXI'

In [60]:
error=pt.pauli_to_bsf(error_pauli)
error

array([1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0])

In [61]:
rng.choice(5, 3)

array([1, 4, 0])

In [62]:
syndrome = pt.bsp(error, code.stabilizers.T)
syndrome

array([1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1])

In [63]:
code.stabilizers.T

array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [38]:
error.T

array([1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 0, 1, 1, 0, 1])

In [41]:
#    3x3 rotated planar code with H or V indicating qubits and hashed/blank plaquettes indicating X/Z stabilizers:
#     ::

#            /---\
#            |   |
#            H6--V7--H8\
#            |###|   |##|
#            |###|   |##|
#            |###|   |##|
#         /--V3---H4---V5-/
#         |##|   |###|
#         |##|   |###|
#         |##|   |###|
#         \--H0---V1---H2
#                |   |
#                \---/

rng = np.random.default_rng() if rng is None else rng
size=3
code=RotatedPlanarCode(*(size,size)) 

n_qubits =code.n_k_d[0]
probability=0.8
error_model = BiasedDepolarizingErrorModel(bias,'Z')
p=error_model.probability_distribution(probability)

error_pauli = ''.join(rng.choice(
            ('I', 'X', 'Y', 'Z'),
            size=n_qubits,
            p=error_model.probability_distribution(probability)
        ))
error=pt.pauli_to_bsf(error_pauli)
syndrome = pt.bsp(error, code.stabilizers.T)
code.stabilizers.T

NameError: name 'rng' is not defined

In [95]:
hvec=np.random.randint(2,size=(len(error_pauli),1))
hvec

array([[1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0]])

In [96]:
error

array([1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1])

In [97]:
q=len(error_pauli)
for i in range(len(error_pauli)):
    if hvec[i]==1:
        error_temp=error[i]
        error[i]=error[q+i]
        error[q+i]=error_temp

In [37]:
error

NameError: name 'error' is not defined

In [81]:
hvec.T

array([[1, 0, 1, 1, 0, 1, 0, 0, 0]])

In [128]:
paulis = ('I', 'X', 'Y', 'Z')
bias=1000
error_model = BiasedDepolarizingErrorModel(bias,'Z')

prob_dist=error_model.probability_distribution(probability)
op_to_pr = dict(zip(paulis, prob_dist))
f='X'
f = pt.pauli_to_bsf(f)
I, X, Y, Z = pt.pauli_to_bsf(paulis)
n=0.1
e=0.1
s=0.1
w=0.1
# n, e, s, w are in {0, 1} so multiply op to turn on or off
op = (f + (n * Z) + (e * X) + (s * Z) + (w * X)) % 2
op

array([1.2, 0.2])

In [129]:
pt.bsf_to_pauli(op)

'X.6'

In [130]:
pt.bsf_to_pauli(np.array([1, 0]))

'X'

In [131]:
pt.pauli_to_bsf('X')

array([1, 0])

In [132]:
op_to_pr['X']

0.0003996003996003996

In [133]:
op_to_pr = dict(zip(paulis, prob_dist))
op_to_pr

{'I': 0.19999999999999996,
 'X': 0.0003996003996003996,
 'Y': 0.0003996003996003996,
 'Z': 0.7992007992007992}

In [134]:
n_qubits

9

In [136]:
hadamard_vec=np.zeros(n_qubits)
for i in range(n_qubits):
    if i%2:
        hadamard_vec[i]=1

In [137]:
hadamard_vec

array([0., 1., 0., 1., 0., 1., 0., 1., 0.])

In [3]:
import numpy as np
np.random.rand(1,1)

array([[0.55376431]])

In [19]:
count =0
for i in range(1000000):
    x=np.random.rand(1,1)
    if x<=0.5:
        count+=1
count

500239

In [23]:
hadamard_mat=np.zeros((5,5))
np.prod(hadamard_mat.shape)

25

In [27]:
from qecsim.models.rotatedplanar import RotatedPlanarCode, RotatedPlanarMPSDecoder

size=3
code=RotatedPlanarCode(*(size,size)) 
tn_max_r, _ = _rotate_q_index((0, 0), code)
_, tn_max_c = _rotate_q_index((code.site_bounds[0], 0), code)

print(tn_max_r,tn_max_c)

4 4


In [26]:

            def _rotate_q_index(index, code):
                """Convert code site index in format (x, y) to tensor network q-node index in format (r, c)"""
                site_x, site_y = index  # qubit index in (x, y)
                site_r, site_c = code.site_bounds[1] - site_y, site_x  # qubit index in (r, c)
                return code.site_bounds[0] - site_c + site_r, site_r + site_c  # q-node index in (r, c)

In [29]:
code._plaquette_indices

[(1, -1), (0, 0), (1, 1), (0, 2), (-1, 0), (1, 0), (0, 1), (2, 1)]

In [30]:
code.site_bounds

(2, 2)

In [36]:
code.is_z_plaquette((2,0))

True

In [347]:
#    3x3 rotated planar code with H or V indicating qubits and hashed/blank plaquettes indicating X/Z stabilizers:
#     ::

#            /---\
#            |   |
#            H6--V7--H8\
#            |###|   |##|
#            |###|   |##|
#            |###|   |##|
#         /--V3---H4---V5-/
#         |##|   |###|
#         |##|   |###|
#         |##|   |###|
#         \--H0---V1---H2
#                |   |
#                \---/

import collections
import itertools
import numpy as np
from qecsim import paulitools as pt
import matplotlib.pyplot as plt
import qecsim
from qecsim import app
from qecsim.models.generic import PhaseFlipErrorModel,DepolarizingErrorModel,BiasedDepolarizingErrorModel
#from qecsim.models.planar import PlanarCode, PlanarMPSDecoder
from qecsim.models.rotatedplanar import RotatedPlanarCode

from _planarmpsdecoder_def import PlanarMPSDecoder_def
from _rotatedplanarmpsdecoder_def import RotatedPlanarMPSDecoder_def
import app_def
import _rotatedplanarmpsdecoder_def
import importlib as imp
imp.reload(app_def)
imp.reload(_rotatedplanarmpsdecoder_def)
import os, time
import multiprocessing as mp
from functools import partial

import collections
import itertools
import json
import logging
import statistics
import time
import numpy as np
from qecsim import paulitools as pt
from qecsim.error import QecsimError
from qecsim.model import DecodeResult

rng = np.random.default_rng(10)

size=3
code=RotatedPlanarCode(*(size,size)) 
bias=100
n_qubits =code.n_k_d[0]
probability=0.2
error_model = BiasedDepolarizingErrorModel(bias,'Z')
p=error_model.probability_distribution(probability)

error_pauli = ''.join(rng.choice(
            ('I', 'X', 'Y', 'Z'),
            size=n_qubits,
            p=error_model.probability_distribution(probability)
        ))
error=pt.pauli_to_bsf(error_pauli)
syndrome = pt.bsp(error, code.stabilizers.T)
syndrome

array([0, 0, 0, 0, 1, 1, 1, 0])

In [348]:
code.stabilizers.T


array([[0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0]])

In [349]:
error_pauli

'ZIZIIIIZI'

In [98]:
code.site_bounds

(2, 2)

In [99]:
int(2*0.6)

1

In [100]:
pH=0.5

In [356]:

size=3
code=RotatedPlanarCode(*(size,size+1)) 

hadamard_mat=np.zeros((3,4))
from sklearn.utils.random import sample_without_replacement

def random_coords(dims, nsamp):
    idx = sample_without_replacement(np.prod(dims), nsamp)
    return np.vstack(np.unravel_index(idx, dims)).T

rand_coords= random_coords(hadamard_mat.shape,int(np.prod(hadamard_mat.shape)*pH))
for row, col in rand_coords:
    hadamard_mat[row,col]=1

In [357]:
hadamard_mat

array([[0., 1., 0., 0.],
       [0., 1., 1., 1.],
       [1., 0., 0., 1.]])

In [360]:
n_qubits = code.n_k_d[0]
hadamard_vec=np.zeros(n_qubits)

Nx=code.site_bounds[0]+1
Ny=code.site_bounds[1]+1

for i,j in np.ndindex(hadamard_mat.shape):
    if hadamard_mat[i,j]==1:
        hadamard_vec[i+j*(Nx-1)]=1
        
hadamard_vec

array([0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1.])

In [118]:
code=RotatedPlanarCode(*(size,size+1)) 
n_qubits =code.n_k_d[0]
hadamard_vec=np.zeros(n_qubits)

hadamard_mat=np.zeros((3,4))
for row, col in np.ndindex(hadamard_mat.shape):
    if (row+col)%2==0:
        hadamard_mat[row,col]=1
for i in range(n_qubits):
    if i%2==0:
        hadamard_vec[i]=1

In [119]:
hadamard_mat

array([[1., 0., 1., 0.],
       [0., 1., 0., 1.],
       [1., 0., 1., 0.]])

In [120]:
hadamard_vec

array([1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.])

In [123]:
hadamard_mat=np.zeros((3,4))
rand_coords= random_coords(hadamard_mat.shape,int(np.prod(hadamard_mat.shape)*pH))
for row, col in rand_coords:
        hadamard_mat[row,col]=1

In [124]:
hadamard_mat

array([[0., 0., 1., 1.],
       [1., 0., 1., 0.],
       [1., 1., 0., 0.]])

In [125]:
rand_coords

array([[2, 0],
       [0, 3],
       [1, 0],
       [2, 1],
       [0, 2],
       [1, 2]])

In [126]:
dims=(3,2)
np.prod(dims)

6

In [130]:
y=random_coords((3,3),4)

In [129]:
x=random_coords((3,3),9)

In [137]:
(3,4) in x

False

In [139]:
error_pauli = ''.join(rng.choice(
        ('I', 'X', 'Y', 'Z'),
        size=n_qubits,
        p=error_model.probability_distribution(probability)
    ))

In [140]:
error_pauli

'ZZIIIZZIIIZI'

In [147]:
x=['X','Y','Z']
x

['X', 'Y', 'Z']

In [149]:
''.join(x)

'XYZ'

In [180]:
x=random_coords(10,2)
y=random_coords(10,10)

x

array([[6],
       [4]])

In [181]:
y

array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8],
       [9]])

In [182]:
[5] in x

False

In [192]:
def complement(listA, listB):
    listC=[]    
    for i in range(len(listA)):
        if listA[i] not in listB:
            listC.append(listA[i][0])
    return listC

In [193]:
z=complement(y,x)

In [194]:
z

[0, 1, 2, 3, 5, 7, 8, 9]

In [212]:
from random import shuffle
x = ['X','Y','Z','I','X','Y','Z']
shuffle(x)
''.join(x)

'ZYIXYZX'

In [199]:
x

[0, 4, 1, 3, 8, 9, 2, 5, 6, 7]

In [208]:
pI=0.2;p=(1-pI)/3
y=[]

In [209]:
for i in range(int(n_qubits*pI)):
    y.append('I')
  
  
        
for i in range(int(n_qubits*p)):
    y.append('X')

for i in range(int(n_qubits*p)):
    y.append('Y')
for i in range(int(n_qubits*p)):
    y.append('Z')
  
  
y
   

['I', 'I', 'X', 'X', 'X', 'Y', 'Y', 'Y', 'Z', 'Z', 'Z']

In [210]:
shuffle(y)
y

['X', 'X', 'Z', 'I', 'I', 'Y', 'Z', 'X', 'Z', 'Y', 'Y']

In [245]:
from random import shuffle
def generate(error_model, code, probability, rng=None):
    """
    See :meth:`qecsim.model.ErrorModel.generate`

    Notes:

    * This method delegates to :meth:`probability_distribution` to find the probability of I, X, Y, Z operators on
      each qubit, assuming an IID error model.
    """
    n_qubits = code.n_k_d[0]
    (pI,pX,pY,pZ)=error_model.probability_distribution(probability)
    error_Pauli=[]
    error_Pauli.extend('I'*int(n_qubits*pI))    
    error_Pauli.extend('X'*int(n_qubits*pX))    
    error_Pauli.extend('Y'*int(n_qubits*pY))    
    error_Pauli.extend('Z'*int(n_qubits*pZ))    
    shuffle(error_Pauli)
    print(error_Pauli)
    return pt.pauli_to_bsf(''.join(error_Pauli))

In [246]:
error_Pauli=generate(error_model,code,0.6)
error_Pauli

['I', 'Z', 'I', 'I', 'I', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z']


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1])

In [285]:
code=RotatedPlanarCode(*(5,5)) 

error_model = BiasedDepolarizingErrorModel(0.5,'Z')

n_qubits = code.n_k_d[0]
(pI,pX,pY,pZ)=error_model.probability_distribution(0.8)
error_Pauli=[]
error_Pauli.extend('I'*round(n_qubits*pI))    
error_Pauli.extend('X'*round(n_qubits*pX))    
error_Pauli.extend('Y'*round(n_qubits*pY))    
error_Pauli.extend('Z'*round(n_qubits*pZ))    
error_Pauli.extend('I'*(n_qubits-round(n_qubits*pI)-round(n_qubits*pX)-round(n_qubits*pY)-round(n_qubits*pZ)))
print(error_Pauli)
for i in range(n_qubits):
    if hadamard_vec[i]==1:
        if error_Pauli[i]=='X':
            error_Pauli[i]='Z'
        elif error_Pauli[i]=='Z':
            error_Pauli[i]='X'
            
print(error_Pauli)


['I', 'I', 'I', 'I', 'I', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z']
['I', 'I', 'I', 'I', 'I', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'X', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Z', 'X', 'X', 'X', 'X', 'X', 'Z']


In [283]:
hadamard_vec=np.random.randint(0,2,n_qubits)

In [284]:
hadamard_vec

array([0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1])

In [275]:
error_Pauli[3]='Z'

In [276]:
error_Pauli

['I', 'I', 'X', 'Z', 'X', 'Y', 'Y', 'Y', 'Z', 'Z', 'Z', 'I']

In [299]:
n_qubits = code.n_k_d[0]
(pI,pX,pY,pZ)=error_model.probability_distribution(0.8)
error_Pauli=[]
error_Pauli.extend('X'*round(n_qubits*pX))    
error_Pauli.extend('Y'*round(n_qubits*pY))    
error_Pauli.extend('Z'*round(n_qubits*pZ))    
error_Pauli.extend('I'*(n_qubits-len(error_Pauli)))
len(error_Pauli)== n_qubits

True

In [297]:
len(error_Pauli)

27

In [295]:
n_qubits

25

In [296]:
error_Pauli.extend('I'*-1)


In [311]:
error_probability_min, error_probability_max = 0.05, 0.5
error_probabilities = np.linspace(error_probability_min, error_probability_max, 18)

In [312]:
sizes

range(5, 9, 2)

In [346]:
def removedup(vec):
    vec2 = [] 
    [vec2.append(x) for x in vec if x not in vec2] 
    return np.array(vec2)

n_qubits=sizes[0]**2
vround = np.vectorize(round)
n_error_vec=removedup(vround(error_probabilities*n_qubits))

n_errors =[]
n_errors.extend(n_error_vec)

for i in range(1,len(sizes)):
    n_qubits=sizes[i]**2
    n_error_vec=removedup(vround(error_probabilities*n_qubits))
    n_errors.extend(n_error_vec[n_error_vec>n_errors[-1]])
    
n_errors

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 21, 22, 23, 24]

In [342]:
n_error_vec[n_error_vec>9]

TypeError: '>' not supported between instances of 'list' and 'int'

In [319]:
round(1.5)

2

In [320]:
vfunc = np.vectorize(round)
vfunc([0.3,2.5])

array([0, 2])

In [322]:
error_probabilities*sizes[0]

array([0.25      , 0.38235294, 0.51470588, 0.64705882, 0.77941176,
       0.91176471, 1.04411765, 1.17647059, 1.30882353, 1.44117647,
       1.57352941, 1.70588235, 1.83823529, 1.97058824, 2.10294118,
       2.23529412, 2.36764706, 2.5       ])

In [324]:
n_errors =[]
vround = np.vectorize(round)

n_qubits=sizes[0]**2
n_errors.extend(vround(error_probabilities*n_qubits))

In [325]:
n_errors

[1, 2, 3, 3, 4, 5, 5, 6, 7, 7, 8, 9, 9, 10, 11, 11, 12, 12]

In [361]:
max_site_x, max_site_y = code.site_bounds
for code_index in itertools.product(range(-1, max_site_x + 1), range(-1, max_site_y + 1)):
    print(code_index)

(-1, -1)
(-1, 0)
(-1, 1)
(-1, 2)
(0, -1)
(0, 0)
(0, 1)
(0, 2)
(1, -1)
(1, 0)
(1, 1)
(1, 2)
(2, -1)
(2, 0)
(2, 1)
(2, 2)
(3, -1)
(3, 0)
(3, 1)
(3, 2)


In [378]:
code=PlanarCode(*(5,5)) 
rng = np.random.default_rng(59)
bias=100

error_model = BiasedDepolarizingErrorModel(bias,'Z')
chi_val=10
decoder = PlanarMPSDecoder_def(chi=chi_val)
error = error_model.generate(code, error_probability_max, rng)
syndrome = pt.bsp(error, code.stabilizers.T)
sample_pauli = decoder.sample_recovery(code, syndrome)
hadamard_mat_sample=np.zeros((2*sample_pauli.code.size[0] - 1, 2*sample_pauli.code.size[1] - 1))
hadamard_mat=np.zeros(hadamard_mat_sample.shape)

n_qubits =code.n_k_d[0]

        
for row, col in np.ndindex(hadamard_mat.shape):
    if row%2==0 and col%2==0:
        hadamard_mat[row,col]=1
# for i in range(n_qubits):
#     if i<sizes[code_index]**2
#         hadamard_vec[i]=1
n_qubits = code.n_k_d[0]
hadamard_vec=np.zeros(np.prod(hadamard_mat.shape))

for i,j in np.ndindex(hadamard_mat.shape):
    if hadamard_mat[i,j]==1:
        hadamard_vec[j+i*hadamard_mat.shape[1]]=1
        
hadamard_vec2=[]        
for row, col in np.ndindex(hadamard_mat.shape):
    if (row%2==0 and col%2==0):
        hadamard_mat[row,col]=1
        hadamard_vec2.append(hadamard_mat[row,col])

for row, col in np.ndindex(hadamard_mat.shape):
    if (row%2==1 and col%2==1):
        hadamard_vec2.append(hadamard_mat[row,col])
        
        
hadamard_vec2=np.array(hadamard_vec2)        

In [379]:
hadamard_vec

array([1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.])

In [380]:
hadamard_vec2

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])

In [371]:
code=RotatedPlanarCode(*(5,5)) 
rng = np.random.default_rng(59)
bias=100
sizes=[5]
error_model = BiasedDepolarizingErrorModel(bias,'Z')
chi_val=10
hadamard_mat=np.zeros((sizes[0],sizes[0]))


n_qubits =code.n_k_d[0]
       
for row, col in np.ndindex(hadamard_mat.shape):
    if (row+col)%2==0:
        hadamard_mat[row,col]=1
        
# for i in range(n_qubits):
#     if i<sizes[code_index]**2
#         hadamard_vec[i]=1
n_qubits = code.n_k_d[0]
hadamard_vec=np.zeros(np.prod(hadamard_mat.shape))

for i,j in np.ndindex(hadamard_mat.shape):
    if hadamard_mat[i,j]==1:
        hadamard_vec[j+i*hadamard_mat.shape[1]]=1

In [372]:
hadamard_vec

array([1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1.])